In [1]:
from dolfin import *

\begin{align}
-\Delta u = \lambda u, \quad \Omega\\
u = 0, \quad \partial \Omega
\end{align}

In [3]:
mesh = UnitSquareMesh(32, 32)
V = FunctionSpace(mesh, "Lagrange", 1)
def boundary(x):
    return x[0] < DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS or x[1] < DOLFIN_EPS or x[1] > 1.0 - DOLFIN_EPS

v0 = Constant(0.0)
bc = DirichletBC(V, u0, boundary)

In [4]:
u = TrialFunction(V)
v = TestFunction(V)

In [7]:
dummy = inner(Constant(1), v)*dx
a = inner(grad(u), grad(v))*dx
asm = SystemAssembler(a, dummy, bc)
A = PETScMatrix(); asm.assemble(A)

In [8]:
b = inner(u, v)*dx
asm = SystemAssembler(b, dummy)
B = PETScMatrix(); asm.assemble(B)
bc.zero(B)

Calling FFC just-in-time (JIT) compiler, this may take some time.


In [16]:
num_eigenfunctions = 15

In [17]:
%%time
solver = SLEPcEigenSolver(A, B)
solver.parameters["solver"] = "krylov-schur"
solver.parameters["spectrum"] = "target magnitude"
solver.parameters["problem_type"] = "gen_hermitian"
solver.parameters["spectral_transform"] = "shift-and-invert"
solver.parameters["spectral_shift"] = 10.
solver.solve(num_eigenfunctions)

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 16 ms


In [26]:
solver.get_eigenpair(0)

(19.786792290191517,
 0.0,
 <dolfin.cpp.la.PETScVector at 0x7fbdd5c051c0>)

In [15]:
eigenmodes = File("eigenmodes.pvd")
eigenfunction = Function(V, name="Eigenfunction")
print(solver.get_number_converged())
for i in range(solver.get_number_converged()):
    (r, _, rv, _) = solver.get_eigenpair(i)
    eigenfunction.vector().zero()
    eigenfunction.vector().axpy(1, rv)
    eigenmodes << eigenfunction

15
